In [5]:
import re
import pickle
import pandas as pd
import numpy as np
import os
import json

### Building data frame from composition file

In [14]:
year_sp = []
factionID = []
speech_id = []
count = 0

with open(f'proc_data/topics30_composition.txt', 'r', encoding='UTF-8') as f:
    
    for line in f:
        
        data = line.strip().split("\t")
        topics = data[2:]
        sp_id = data[1] 
        
        #find position of sp_id
        sp_id_index = [i for i, e in enumerate(sp_id) if e == '_']
        
        #slincing for sp_id
        sp_ID = sp_id[sp_id_index[0]+1:sp_id_index[1]]
        speech_id.append(sp_ID)
        
        #find position of year
        year_index = [i for i, e in enumerate(sp_id) if e == '/']
        
        #slicing for year
        year = int(sp_id[year_index[7]+1:year_index[8]])
        year_sp.append(year)
        
        #find position of faction id
        faction_index = [i for i, e in enumerate(sp_id) if e == '_']

        #slicing for faction id
        faction_id = int(sp_id[faction_index[2]+1:-4])
        factionID.append(faction_id)
        
        topics = [float(t) for t in topics]

In [15]:
columns = ["index", "filename"] 

sum_topics = []

for i in range(30):
    columns.append(f'topic{i}')
    
df = pd.read_csv(f'proc_data/topics30_composition.txt', sep="\t", names=columns)

# adding speechID, year and faction
#df.insert(loc, column_name, value)
df.insert(2, "speechID", speech_id)
df.insert(3, "year", year_sp)
df.insert(4, "faction", factionID)


for i in range(30):
    sum_topics.append(sum(df[f'topic{i}']))

In [16]:
df = df.drop(columns=["filename"])

In [17]:
df

,index,speechID,year,faction,topic0,topic1,topic2,topic3,topic4,topic5,...,topic20,topic21,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29
0,0,1004,1949,-1,0.001572,0.001572,0.001572,0.001572,0.001572,0.001572,...,0.029874,0.001572,0.001572,0.001572,0.001572,0.001572,0.001572,0.001572,0.001572,0.001572
1,1,100,1949,2,0.064208,0.000273,0.000273,0.018306,0.487158,0.000273,...,0.000273,0.093716,0.000273,0.000273,0.000273,0.000273,0.000273,0.000273,0.000273,0.000273
2,2,1011,1949,23,0.000634,0.000634,0.000634,0.004436,0.171736,0.000634,...,0.000634,0.008238,0.000634,0.000634,0.000634,0.000634,0.107098,0.000634,0.008238,0.000634
3,3,1013,1949,26,0.001984,0.001984,0.013889,0.001984,0.192460,0.001984,...,0.001984,0.025794,0.001984,0.001984,0.001984,0.001984,0.001984,0.001984,0.001984,0.001984
4,4,1021,1949,4,0.014056,0.002008,0.026104,0.002008,0.495984,0.002008,...,0.002008,0.002008,0.002008,0.002008,0.002008,0.002008,0.014056,0.002008,0.002008,0.002008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247452,247452,1060943,2021,0,0.000526,0.000526,0.000526,0.000526,0.000526,0.038381,...,0.022608,0.375920,0.000526,0.000526,0.006835,0.000526,0.000526,0.249737,0.073081,0.003680
247453,247453,1060945,2021,23,0.000744,0.000744,0.000744,0.000744,0.000744,0.165923,...,0.000744,0.201637,0.000744,0.005208,0.005208,0.000744,0.036458,0.000744,0.197173,0.005208
247454,247454,1060947,2021,4,0.000709,0.017730,0.000709,0.004965,0.000709,0.179433,...,0.000709,0.417730,0.000709,0.000709,0.000709,0.000709,0.034752,0.000709,0.192199,0.000709
247455,247455,1060949,2021,23,0.018788,0.004242,0.000606,0.000606,0.000606,0.076970,...,0.000606,0.226061,0.000606,0.000606,0.000606,0.000606,0.000606,0.229697,0.215152,0.000606


### Filter out speeches where topic5 is the most prevalent

In [18]:
IMPORTANT_TOPIC = 'topic5'

In [19]:
TOPICS = [f'topic{i}' for i in range(30)]

In [20]:
def insert_max(df: pd.DataFrame, topics: list[str]) -> pd.DataFrame:
    values = df[topics]
    max_values = values.max(1)  # max(axis = 1) -> max in row
    df['maxValue'] = max_values
    return df

def filter_prevalent(df: pd.DataFrame, topic: str) -> pd.DataFrame:
    df['baseTopic'] = topic
    return df[df[topic] >= df['maxValue']]
    

In [21]:
FILTERED = filter_prevalent(insert_max(df, TOPICS), IMPORTANT_TOPIC)

In [22]:
FILTERED

,index,speechID,year,faction,topic0,topic1,topic2,topic3,topic4,topic5,...,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29,maxValue,baseTopic
798,798,2028,1950,20,0.021253,0.001119,0.001119,0.001119,0.135347,0.262864,...,0.001119,0.001119,0.001119,0.014541,0.001119,0.014541,0.001119,0.001119,0.262864,topic5
1103,1103,2791,1950,4,0.026104,0.002008,0.014056,0.002008,0.279116,0.363454,...,0.002008,0.002008,0.002008,0.002008,0.002008,0.002008,0.002008,0.002008,0.363454,topic5
1440,1440,3680,1950,-1,0.030528,0.000825,0.000825,0.005776,0.238449,0.268152,...,0.000825,0.000825,0.000825,0.000825,0.000825,0.005776,0.000825,0.000825,0.268152,topic5
2008,2008,4989,1950,-1,0.001938,0.001938,0.013566,0.001938,0.036822,0.327519,...,0.013566,0.001938,0.001938,0.001938,0.001938,0.001938,0.001938,0.001938,0.327519,topic5
3101,3101,7563,1950,-1,0.000358,0.011087,0.023963,0.000358,0.152718,0.300787,...,0.000358,0.000358,0.000358,0.013233,0.000358,0.000358,0.000358,0.008941,0.300787,topic5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246996,246996,1059715,2021,4,0.006744,0.000963,0.012524,0.012524,0.000963,0.492293,...,0.000963,0.000963,0.029865,0.006744,0.000963,0.087669,0.081888,0.000963,0.492293,topic5
247012,247012,1059751,2021,0,0.000689,0.000689,0.029614,0.000689,0.062672,0.261019,...,0.008953,0.000689,0.004821,0.000689,0.079201,0.029614,0.215565,0.000689,0.261019,topic5
247134,247134,1060129,2021,6,0.002110,0.002110,0.002110,0.002110,0.002110,0.483122,...,0.002110,0.002110,0.002110,0.002110,0.002110,0.179325,0.002110,0.002110,0.483122,topic5
247317,247317,1060584,2021,6,0.005636,0.000805,0.000805,0.000805,0.005636,0.237520,...,0.010467,0.000805,0.010467,0.000805,0.179549,0.005636,0.126409,0.005636,0.237520,topic5


In [23]:
def make_speech_data(row: pd.Series, neighbours_descending: list[str]) -> dict[list[str]]:
    data = {
        'index': str(row['index']), 
        'id': str(row['speechID']),
        'year': str(row['year']),
        'faction': str(row['faction']),
        'baseTopic': str(row['baseTopic']),
        'neighbours': neighbours_descending,
    }
    return data


def get_neighbours(row: pd.Series, topics: list[str], k: int) -> list[str]:
    assert k <= len(topics) - 1, f'Number of neighbours to search: {k} must be less than number of topics: {len(topics)}'
    values_sorted = row[topics].sort_values(ascending=False)
    sorted_topic_names = values_sorted.index.to_list()
    return sorted_topic_names[1:k + 1]  # exclude the first topic 


def parse(df: pd.DataFrame, topics: list[str], k: int) -> list[list[str]]:
    result = []
    for _, series in df.iterrows():  # note (_, value) iteration 
        neighbours = get_neighbours(series, topics, k)
        speech_data = make_speech_data(series, neighbours)
        result.append(speech_data)
    return result

In [24]:
K = 3  

In [25]:
PREVALENT_WITH_NEIGHBOURS_DESCENDING = parse(FILTERED, TOPICS, K)

In [26]:
print(len(PREVALENT_WITH_NEIGHBOURS_DESCENDING))

5943


### Filter for period of investigation
+ Loop through each element and obtain the unique topics next to topic5 
+ sort the topic_counts dictionary by count size (in descending order)

In [27]:
## Filter for period of investigation
filtered_period = [period for period in PREVALENT_WITH_NEIGHBOURS_DESCENDING if period["year"] >= "1970" and period["year"] <= "2005"]

In [28]:
print(len(filtered_period))

2411


In [ ]:
topic_counts_period = {}

for element in filtered_period:
    base_topic = [element['baseTopic']]
    neighbours = element['neighbours']
    element_topics = set(base_topic + neighbours)
    for topic in element_topics:
        if topic in topic_counts_period:
            topic_counts_period[topic] += 1
        else:
            topic_counts_period[topic] = 1

In [ ]:
sorted_counts_period = sorted(topic_counts_period.items(), key=lambda x: x[1], reverse=True)

### Filter by faction

In [142]:
# filtered period by faction -1
filtered_without_faction = [fact for fact in filtered_period if fact["faction"] == "-1"]

In [145]:
print(len(filtered_without_faction))

500


In [146]:
cnt_without_faction = {}  

for item in filtered_without_faction:
    year = item["year"] 
    if year in cnt_without_faction:
        cnt_without_faction[year] += 1  
    else:
        cnt_without_faction[year] = 1  

print(cnt_without_faction)

{'1970': 3, '1971': 5, '1972': 2, '1973': 6, '1974': 9, '1975': 11, '1976': 12, '1977': 7, '1978': 6, '1979': 11, '1980': 10, '1981': 12, '1982': 10, '1983': 6, '1984': 8, '1985': 12, '1986': 9, '1987': 2, '1988': 5, '1989': 20, '1990': 20, '1991': 7, '1992': 24, '1993': 22, '1994': 16, '1995': 18, '1996': 17, '1997': 23, '1998': 22, '1999': 23, '2000': 27, '2001': 18, '2002': 21, '2003': 19, '2004': 42, '2005': 15}


In [29]:
# filtered period by faction
filtered_faction = [fact for fact in filtered_period if fact["faction"] == "23" or fact["faction"] == "4"]

In [30]:
print(len(filtered_faction))

1022


In [19]:
topic_counts_faction = {}

# Loop through each element and obtain the unique topics
for element in filtered_faction:
    base_topic = [element['baseTopic']]
    neighbours = element['neighbours']
    element_topics = set(base_topic + neighbours)
    for topic in element_topics:
        if topic in topic_counts_faction:
            topic_counts_faction[topic] += 1
        else:
            topic_counts_faction[topic] = 1

In [20]:
print(topic_counts_faction)

{'topic4': 78, 'topic5': 1022, 'topic10': 361, 'topic2': 24, 'topic11': 12, 'topic25': 38, 'topic21': 421, 'topic6': 511, 'topic12': 12, 'topic3': 138, 'topic19': 50, 'topic15': 390, 'topic14': 113, 'topic26': 112, 'topic13': 7, 'topic1': 100, 'topic16': 67, 'topic17': 36, 'topic20': 8, 'topic28': 204, 'topic27': 119, 'topic18': 115, 'topic8': 97, 'topic0': 4, 'topic29': 19, 'topic22': 5, 'topic23': 4, 'topic7': 4, 'topic9': 1}


In [21]:
# Sort the topic_counts dictionary by count size
sorted_counts_faction = sorted(topic_counts_faction.items(), key=lambda x: x[1], reverse=True)

In [22]:
print(sorted_counts_faction)

[('topic5', 1022), ('topic6', 511), ('topic21', 421), ('topic15', 390), ('topic10', 361), ('topic28', 204), ('topic3', 138), ('topic27', 119), ('topic18', 115), ('topic14', 113), ('topic26', 112), ('topic1', 100), ('topic8', 97), ('topic4', 78), ('topic16', 67), ('topic19', 50), ('topic25', 38), ('topic17', 36), ('topic2', 24), ('topic29', 19), ('topic11', 12), ('topic12', 12), ('topic20', 8), ('topic13', 7), ('topic22', 5), ('topic0', 4), ('topic23', 4), ('topic7', 4), ('topic9', 1)]


In [23]:
speech_cnt_faction = {}  

for item in filtered_faction:
    year = item["year"] 
    if year in speech_cnt_faction:
        speech_cnt_faction[year] += 1  
    else:
        speech_cnt_faction[year] = 1  

print(speech_cnt_faction)  

{'1970': 2, '1975': 2, '1976': 5, '1978': 3, '1979': 6, '1980': 3, '1981': 12, '1982': 18, '1983': 7, '1984': 11, '1985': 15, '1986': 28, '1987': 14, '1988': 25, '1989': 46, '1990': 25, '1991': 34, '1992': 47, '1993': 70, '1994': 50, '1995': 32, '1996': 41, '1997': 53, '1998': 41, '1999': 68, '2000': 73, '2001': 61, '2002': 34, '2003': 87, '2004': 69, '2005': 40}


### Filter topics to search for

In [31]:
# filter to all 3
topics_to_search = ["topic14", "topic18", "topic27"]
list_search = [d for d in filtered_faction if any(topic in d["neighbours"] for topic in topics_to_search)]

In [32]:
# if any is 14, 18, 27
print(len(list_search))

322


In [34]:
sp_cdu = []
sp_spd = []

for fact in list_search:
    if fact["faction"] == "23":
        sp_spd.append(fact)
        
    else: 
        sp_cdu.append(fact)

print(len(sp_cdu))
print(len(sp_spd))

195
127


In [26]:
# filter 14
topics_to_search = ["topic14"]
list_search_1 = [d for d in filtered_faction if any(topic in d["neighbours"] for topic in topics_to_search)]

In [34]:
# if any is 14
print(len(list_search_1))

113


In [33]:
print(json.dumps(list_search_1, indent=4))

[
    {
        "index": "61689",
        "id": "258422",
        "year": "1976",
        "faction": "4",
        "baseTopic": "topic5",
        "neighbours": [
            "topic15",
            "topic14",
            "topic21"
        ]
    },
    {
        "index": "62320",
        "id": "261768",
        "year": "1976",
        "faction": "4",
        "baseTopic": "topic5",
        "neighbours": [
            "topic6",
            "topic26",
            "topic14"
        ]
    },
    {
        "index": "67438",
        "id": "294345",
        "year": "1978",
        "faction": "4",
        "baseTopic": "topic5",
        "neighbours": [
            "topic26",
            "topic14",
            "topic4"
        ]
    },
    {
        "index": "67716",
        "id": "295936",
        "year": "1979",
        "faction": "23",
        "baseTopic": "topic5",
        "neighbours": [
            "topic6",
            "topic2",
            "topic14"
        ]
    },
    {
        "index": "7

In [106]:
# filter 18
topics_to_search = ["topic18"]
list_search_2 = [d for d in filtered_faction if any(topic in d["neighbours"] for topic in topics_to_search)]

In [107]:
# if any is 18
print(len(list_search_2))

115


In [30]:
# filter 27
topics_to_search = ["topic27"]
list_search_3 = [d for d in filtered_faction if any(topic in d["neighbours"] for topic in topics_to_search)]

In [31]:
# if any is 27
print(len(list_search_3))

119


### Search speech

In [26]:
list_cnt = {}  

for item in list_search:
    year = item["year"] 
    if year in list_cnt:
        list_cnt[year] += 1  
    else:
        list_cnt[year] = 1  

print(list_cnt)  

{'1976': 2, '1978': 1, '1979': 1, '1980': 2, '1981': 4, '1982': 4, '1983': 1, '1984': 3, '1985': 4, '1986': 10, '1987': 7, '1988': 9, '1989': 13, '1990': 9, '1991': 16, '1992': 16, '1993': 21, '1994': 11, '1995': 11, '1996': 14, '1997': 17, '1998': 13, '1999': 18, '2000': 23, '2001': 17, '2002': 9, '2003': 30, '2004': 19, '2005': 17}


In [27]:
print(json.dumps(list_search, indent=4))

[
    {
        "index": "61689",
        "id": "258422",
        "year": "1976",
        "faction": "4",
        "baseTopic": "topic5",
        "neighbours": [
            "topic15",
            "topic14",
            "topic21"
        ]
    },
    {
        "index": "62320",
        "id": "261768",
        "year": "1976",
        "faction": "4",
        "baseTopic": "topic5",
        "neighbours": [
            "topic6",
            "topic26",
            "topic14"
        ]
    },
    {
        "index": "67438",
        "id": "294345",
        "year": "1978",
        "faction": "4",
        "baseTopic": "topic5",
        "neighbours": [
            "topic26",
            "topic14",
            "topic4"
        ]
    },
    {
        "index": "67716",
        "id": "295936",
        "year": "1979",
        "faction": "23",
        "baseTopic": "topic5",
        "neighbours": [
            "topic6",
            "topic2",
            "topic14"
        ]
    },
    {
        "index": "7

In [26]:
with open('raw_data/speeches_fixed.csv', 'r', encoding='UTF-8') as fr:
    head = next(fr)
    print(head)
    for line in fr:
        data = line.strip().split("|")
        #print(data)
        #break
        sp_id = data[0]
        #print(type(sp_id))
        #faction = data[7]
        #year = data[11][:4]
        speech = data[6]
        if sp_id == '655667':
            #print('it is working')
            print(data[0:6])
            print(data[7:12])
            print(speech)
            

id|session|electoralTerm|firstName|lastName|politicianId|speechContent|factionId|documentUrl|positionShort|positionLong|date

['655667', '82', '15', 'Wolfgang', 'Thierse', '11002318']
['23', 'https://dip21.bundestag.de/dip21/btp/15/15082.pdf', 'Member of Parliament', '', '2003-12-11']
Liebe Kolleginnen und Kollegen! Für diese Debatte  haben wir schlechten Anlass. Auch nach fünf Jahrzehnten des demokratischen Konsenses und der historischen Aufklärung ist der Antisemitismus in Deutschland nicht überwunden; schlimmer noch: Antisemitische Ressentiments sind keine Randerscheinung, sondern entfalten ihre fatale Wirkung auch in der Mitte unserer Gesellschaft bis hinein in dieses Hohe Haus. Lassen Sie mich gerade wegen dieses Anlasses sagen: Es ist gut, dass wir diese Debatte führen. Noch besser ist es, dass alle Fraktionen des Deutschen Bundestages ohne Ausnahme für diese Debatte eingetreten sind und sie gemeinsam eine Entschließung gegen den Antisemitismus erarbeitet und zur Abstimmung einge